##Алгоритм выполнения:
1.Строю эмбеддинги слов из списка(название + описание)  

2.Выделяю сущности из списка текстов

3.Беру контекст найденной сущности(окно = 15 слов) и создаю  эмбеддинг

4.С помощью индексов FAISS по метрике косинусово расстояние ищу совпадения по смыслу

5.Далее провожу ранжировку с помощью кросс-энкодера

6.Делю на primary/mention

In [ ]:
!pip install langdetect
!pip install emoji
!python -m spacy download ru_core_news_lg
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import spacy
import json
import emoji
import torch
import faiss
import numpy as np
import logging
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, models, CrossEncoder
from concurrent.futures import ThreadPoolExecutor

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_items = '/content/drive/MyDrive/test_items.csv'
path_texts = '/content/drive/MyDrive/test_texts.csv'

In [ ]:
df_items = pd.read_csv(path_items,header=None)
df_texts =  pd.read_csv(path_texts,header=None)


In [ ]:
df_items.columns = ['id','items','script']
print(df_items.head(10))

      id                    items  \
0  29351              Чёрная дыра   
1  13055                 DAIHATSU   
2  24479             Норма Дюмонт   
3   2474                      KPI   
4  24740                     ХМАО   
5  19997          Денис Никифоров   
6   5144  Калининградская область   
7  22406            Наталья Ветер   
8  26336             Артур Каюмов   
9   5204              Новостройки   

                                              script  
0  Область пространства-времени, гравитационное п...  
1  Daihatsu Motor Co., Ltd.&#32;(яп. ダイハツ工業株式会社 &...  
2  Бразильский боец смешанных единоборств в полул...  
3  Это числовые показатели деятельности, которые ...  
4  Субъект Российской Федерации. Территориально я...  
5          Российский актёр театра и кино, каскадёр.  
6  Калининградская область — субъект Российской Ф...  
7  Руководитель департамента природных ресурсов и...  
8  Российский хоккеист, нападающий ярославского «...  
9  Дома, которые только что достроены, а к

In [ ]:
df_texts.columns = ['text']
print(df_texts.head())

                                                text
0  <p>О жестокости атамана Бориса Анненкова ходил...
1  ❤️Звенигород Руководитель отделения Андрей Пон...
2  Жителям Казани сообщили, что до конца саммита ...
3  ⚡⚡⚡Радий Хабиров срочно покинул Башкирию Повод...
4  Начните сегодня или когда угодно: бесплатные к...


In [ ]:
# очистка текста
def clean_text(text):
    text = text.lower()

    text = re.sub(r'<.*?>', '', text)

    text = emoji.replace_emoji(text, replace='')

    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'#\S+', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    text = re.sub(r'[«»"]', ' ', text)

    text = re.sub(r'[^\w\s]', ' ', text)

    return text

df_texts['text'] = df_texts['text'].apply(lambda x: clean_text(str(x)))

In [ ]:
df_texts = df_texts.applymap(lambda x: x.strip() if isinstance(x, str) else x)

<ipython-input-23-4ce5c74b8379>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_texts = df_texts.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
device = 0 if torch.cuda.is_available() else -1

In [ ]:
reranker_model = CrossEncoder("DiTy/cross-encoder-russian-msmarco", max_length=512)
semantic_model = SentenceTransformer("DiTy/bi-encoder-russian-msmarco")
nlp = spacy.load("ru_core_news_lg")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
logging.basicConfig(filename='process.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [ ]:
# Функция для нормализации эмбеддингов
def normalize_embeddings(embeddings):
    logger.debug("Нормализация эмбеддингов...")
    embeddings = np.array(embeddings)
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
# Создание эмбеддингов(слово + описание) для слов из списка и индексов FAISS
def load_entities_to_faiss(df_items):
    logger.debug("Загрузка сущностей и создание индекса FAISS...")
    entities = []
    embeddings = []

    for _, row in df_items.iterrows():
        entity = {
            'id': int(row['id']),
            'name': row['items'].lower(),
            'description': row['script'].lower() if pd.notna(row['script']) else ''

        }
        entities.append(entity)

        combined_text = f"{entity['name']} {entity['description']}"
        embedding = semantic_model.encode(combined_text, convert_to_tensor=False)
        embeddings.append(embedding)

    normalized_embeddings = normalize_embeddings(embeddings)

    d = normalized_embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(normalized_embeddings.astype('float32'))

    id_to_index = {i: entity['id'] for i, entity in enumerate(entities)}
    index_to_id = {v: k for k, v in id_to_index.items()}
    return entities, index, normalized_embeddings, id_to_index, index_to_id

In [ ]:
# Ранжирует топ-150 кандидатов и возвращает лучшего только если его скор выше порога.
def rerank_top_candidate(query, candidates, reranker_model, score_threshold=0.85, batch_size=32):

    if not candidates:
        return None

    pairs = [(query, candidate["text"]) for candidate in candidates]

    batched_pairs = [pairs[i:i + batch_size] for i in range(0, len(pairs), batch_size)]

    all_scores = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(reranker_model.predict, batch) for batch in batched_pairs]

        for future in futures:
            all_scores.extend(future.result())


    for i, score in enumerate(all_scores):
        candidates[i]["rerank_score"] = score

    filtered_candidates = sorted(
        (candidate for candidate in candidates if candidate["rerank_score"] >= score_threshold),
        key=lambda x: x["rerank_score"], reverse=True
    )


    if not filtered_candidates:
        return None

    best_candidate = filtered_candidates[0]
    print('Лучший кандидат: ', best_candidate, 'Пары: ', pairs)

    return best_candidate

In [ ]:
def determine_entity_id_with_faiss(entity_name, entities, faiss_index, id_to_index, context, similarity_threshold=0.4, top_k=150):
    """
    Выполняет поиск с Bi-Encoder и FAISS, затем ранжирует топ-15 кандидатов с Cross-Encoder.
    """
    logger.debug(f"Определение лучшего кандидата для '{entity_name}'...")


    combined_input = f"{entity_name} {context}"
    combined_embedding = semantic_model.encode(combined_input, convert_to_tensor=False).reshape(1, -1)
    combined_embedding = normalize_embeddings(combined_embedding)

    distances, indices = faiss_index.search(combined_embedding, k=top_k)

    candidates = []
    for i, index in enumerate(indices[0]):
        if distances[0][i] >= similarity_threshold:
            entity_id = id_to_index.get(index)
            if entity_id is not None:

                candidates.append({
                    "id": entity_id,
                    "text": entities[index]["name"] + " " + entities[index]["description"],
                    "faiss_score": distances[0][i]
                })

    if not candidates:
        logger.debug(f"Нет кандидатов с порогом сходства {similarity_threshold}.")
        return None, None

    best_candidate = rerank_top_candidate(combined_input, candidates, reranker_model,score_threshold=0.85)


    if best_candidate:
        logger.debug(f"Лучший кандидат: ID={best_candidate['id']}, Сходство={best_candidate['rerank_score']:.2f}")
        print(f"Лучший кандидат: ID={best_candidate['id']}, Сходство={best_candidate['rerank_score']:.2f}")
        return best_candidate["id"], best_candidate["rerank_score"]

    return None, None

In [ ]:
# Определение роли сущности в тексте
def classify_entity_role(doc, entity_name):
    for token in doc:
        if token.text.lower() == entity_name:
            if token.dep_ in {'nsubj', 'ROOT', 'obj'}:
                return "primary"

    return "mention"

In [ ]:
def analyze_text(text, entities, faiss_index, embeddings, id_to_index, primary_set, mention_set, context_window=15):
    primary = []
    mentions = []

    doc = nlp(text)

    for ent in doc.ents:
        entity_name = ent.text.lower()
        entity_idx = next((i for i, token in enumerate(doc) if token.text.lower() == entity_name), None)

        if entity_idx is not None:
            context = ' '.join([token.text for token in doc[max(0, entity_idx - context_window):min(len(doc), entity_idx + context_window + 1)]])

            entity_id, similarity = determine_entity_id_with_faiss(entity_name, entities, faiss_index, id_to_index, context)

            if entity_id is not None:
                role = classify_entity_role(doc, entity_name)
                if role == "primary" :
                    primary.append({
                        'entity_id': entity_id,
                        'entity_name': entity_name,
                        'similarity': similarity
                    })
                    primary_set.add(entity_id)
                elif role == "mention" :
                    mentions.append({
                        'entity_id': entity_id,
                        'entity_name': entity_name,
                        'similarity': similarity
                    })
                    mention_set.add(entity_id)

    return primary, mentions

In [ ]:
def format_entities(entities):
    return ', '.join([f"{e['entity_name']} (ID: {e['entity_id']}, Similarity: {e['similarity']:.2f})" for e in entities])

In [ ]:
def separate_entities(primary_set,mention_set):

    entities_json = {
        'primary': list(primary_set),
        'mentions': list(mention_set)
    }

    with open('entities.json', 'w', encoding='utf-8') as json_file:
        json.dump(entities_json, json_file, ensure_ascii=False, indent=4)

    return json.dumps(entities_json, ensure_ascii=False, indent=4)

In [ ]:
def process_texts_and_entities(df_items, df_texts):

    entities, faiss_index, embeddings, id_to_index,index_to_id = load_entities_to_faiss(df_items)

    primary_set = set()
    mention_set = set()

    results = []
    for idx, row in df_texts.iterrows():
        text = row['text']
        primary, mentions = analyze_text(
            text, entities, faiss_index, embeddings, id_to_index, primary_set, mention_set
        )

        results.append({
            'id': idx,
            'text': text,
            'primary': format_entities(primary) if primary else '[]',
            'mentions': format_entities(mentions) if mentions else '[]'
        })


    separate_entities(primary_set, mention_set)
    return results

In [ ]:
results = process_texts_and_entities(df_items, df_texts)

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv('results.csv', index=False)